In [114]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma

In [115]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [116]:
import os
import getpass

In [117]:
!pip install -qU langchain-openai
os.environ["LANGCHAIN_TRACING_V2"] = "false"  
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

 ········


In [118]:
!pip install pypdf
import PyPDF2

pdf_loader = PyPDFLoader(file_path='Resilient_LosAngeles_Plan.pdf') 
docs = pdf_loader.load()

In [119]:
len(docs[0].page_content)

91

In [120]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

468

In [121]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [122]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What Climate Threats?")

len(retrieved_docs)

6

In [123]:
print(retrieved_docs[0].page_content)

Open Space
  Disparities in Access to Transit
  Food Insecurity
Climate Adaptation
Los Angeles is taking action to reduce the impact of future climate change, while also  
preparing for and adapting to the already-changing environment. Over the past few years, Los Angeles has experienced a new normal, with record-breaking drought, heat, and storms; climate projections illustrate the future challenges with urban heat island, unpredictable snowpack, and sea level rise.
  Climate Change
  Air Quality/Pollution
  Urban Heat Island
  Extreme Heat  Drought
  Severe Weather and Flooding
  Sea Level Rise
  Extreme Cold
Infrastructure Modernization
Los Angeles has a vast, complex, and aging infrastructure. The City is investing billions of dollars to restore, rebuild, and modernize the aging infrastructure to prevent existing infrastructure from failing and to meet the needs of a large and dynamic city.
  Aging Infrastructure  Infrastructure or Building FailureShock Stress


In [124]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [128]:
definitions = {
    "Climate Change": "A long-term shift in weather patterns and temperatures, primarily caused by human activities emitting greenhouse gases (GHGs).",
    "Greenhouse Gases (GHGs)": "Atmospheric gases like CO₂, CH₄, N₂O that absorb and emit radiation, leading to the greenhouse effect.",
    "Anthropogenic Emissions": "Emissions of GHGs resulting from human activities such as burning fossil fuels and deforestation.",
    "Climate Impacts": "Consequences of climate-related hazards on natural and human systems, affecting lives, ecosystems, economies, and infrastructure.",
    "Climate Risk": "Potential negative consequences from climate impacts, resulting from the interaction of hazard, exposure, and vulnerability.",
    "Climate Vulnerability": "The degree to which a system is susceptible to harm from climate change and its ability to adapt.",
    "Climate Policies": "Strategies and measures adopted to implement resilience, mitigation, and adaptation options.",
    "Resilience": "The ability of systems to cope with climate hazards by maintaining essential functions and adapting to changes.",
    "Resilience Options": "Strategies to build resilience through policy changes, infrastructure improvements, planning, etc.",
    "Mitigation": "Efforts to reduce or prevent emission of GHGs.",
    "Mitigation Options": "Technologies or practices that contribute to mitigation, like renewable energy or waste minimization.",
    "Adaptation": "Adjusting systems to actual or expected climate changes to minimize harm or exploit beneficial opportunities.",
    "Adaptation Options": "Strategies addressing climate change adaptation, including structural, institutional, ecological, and behavioral measures.",
    "Climate Justice": "Ensuring equitable sharing of the burdens and benefits of climate change impacts.",
    "Maladaptation": "Actions that may increase vulnerability to climate change or diminish resilience.",
    "Scenario": "A plausible description of how the future may develop based on a coherent set of assumptions."
}

In [129]:
normalized_definitions = {key.lower(): value for key, value in definitions.items()}
definitions_context = "\n".join([f"{key}: {value}" for key, value in definitions.items()])


In [130]:
system_prompt = (
    "You are a researcher specializing in extracting climate-related information from climate adaptation and resilience plans."
    "You are provided with a document (referred to as the 'plan') from a state or city in the United States (the 'planning jurisdiction')that has adopted policies addressing climate change, which may pertain to resilience, mitigation, and adaptation."
    "Please provide a detailed answer to the following question and include citations in the format: (Chapter name, Section header if applicable, Page number). If any of these citation variables are missing, write N/A"
    "\n\n"
    "{context}"
    "Definitions:\n"
    f"{definitions_context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
print("Prompt Structure:", prompt)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


Prompt Structure: input_variables=['context', 'input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are a researcher specializing in extracting climate-related information from climate adaptation and resilience plans.You are provided with a document (referred to as the 'plan') from a state or city in the United States (the 'planning jurisdiction')that has adopted policies addressing climate change, which may pertain to resilience, mitigation, and adaptation.Please provide a detailed answer to the following question and include citations in the format: (Chapter name, Section header if applicable, Page number). If any of these citation variables are missing, write N/A\n\n{context}Definitions:\nClimate Change: A long-term shift in weather patterns and temperatures, primarily caused by human activities emitting greenhouse gases (GHGs).\nGreenhouse Gases (GHGs)

In [131]:
import pandas as pd
questions_df = pd.read_csv("Question_Prompts.csv")

# Extract the 'Question' column (or whichever column contains your questions)
questions = questions_df['Question'].tolist()

In [228]:
question_1 = questions[0]
response_1 = rag_chain.invoke({"input": question_1.lower()})
output_1 = f"### Output for Question 1\n\n**{question_1}**\n{response_1['answer']}\n"
print(f"### Output for Question 1\n\n**{question_1}**\n{response_1['answer']}\n")

### Output for Question 1

**Provide a summary of information available from the plan regarding the planning jurisdiction, including its name, location, population, geography, major industries, per capita income, and similar information.**
N/A



In [224]:
question_2 = questions[1]
response_2 = rag_chain.invoke({"input": question_2.lower()})
output_2 = f"### Output for Question 2\n\n**{question_2}**\n{response_2['answer']}\n"
print(f"### Output for Question 2\n\n**{question_2}**\n{response_2['answer']}\n")

### Output for Question 2

**To what extent does the plan address resilience, mitigation, and adaptation?**
The plan addresses resilience, mitigation, and adaptation through a multifaceted approach that incorporates various strategies and guiding principles aimed at enhancing the city’s ability to cope with climate-related challenges.

1. **Resilience**: The plan emphasizes the importance of resilience by proposing the inclusion of a guiding principle in the General Plan that focuses on land-use decisions. This principle aims to enhance the recovery capabilities of individuals, neighborhoods, and economic systems from disasters, climate change impacts, and economic shifts (N/A). The plan also outlines specific areas of resilience, including Energy, Transportation and Land Use, Ocean and Coastal Resources, Water, and Public Health, indicating a comprehensive approach to resilience planning (N/A).

2. **Mitigation**: While the plan primarily focuses on adaptation and resilience, it impli

In [234]:
question_3 = questions[2]
response_3 = rag_chain.invoke({"input": question_3.lower()})
output_3 = f"### Output for Question 3\n\n**{question_3}**\n{response_3['answer']}\n"
print(f"### Output for Question 3\n\n**{question_3}**\n{response_3['answer']}\n")

### Output for Question 3

**Does the plan describe a scenario of climate change the planning jurisdiction will face and for which it is planning climate policies for resilience, mitigation, and adaptation? If so, please summarize any such scenario.**
The provided text does not explicitly describe a specific scenario of climate change that the planning jurisdiction will face. Instead, it emphasizes the need for the General Plan to incorporate climate adaptation, hazard mitigation, and recovery strategies aimed at improving resilience against various shocks and stresses, including those resulting from climate change, economic shifts, and disasters. The focus is on encouraging land-use decisions that enhance the ability of individuals and communities to recover from such events, although specific climate scenarios or projected impacts are not detailed.

The text mentions that the Framework addresses resilience in areas such as Energy, Transportation and Land Use, Ocean and Coastal Resour

In [235]:
question_4 = questions[3]
response_4 = rag_chain.invoke({"input": question_4.lower()})
output_4 = f"### Output for Question 4\n\n**{question_4}**\n{response_4['answer']}\n"
print(f"### Output for Question 4\n\n**{question_4}**\n{response_4['answer']}\n")

### Output for Question 4

**What climate impacts does the plan identify as presenting climate risks for the planning jurisdiction?**
The plan identifies various climate impacts that pose climate risks for the planning jurisdiction, particularly focusing on the following areas:

1. **Energy**: The impact of climate change on energy systems, including the reliability and sustainability of energy supply amidst changing climate conditions.

2. **Transportation and Land Use**: Risks associated with climate impacts on transportation infrastructure and planning, which could affect mobility and accessibility.

3. **Ocean and Coastal Resources**: The implications of rising sea levels, coastal erosion, and increased storm intensity that threaten coastal ecosystems and communities.

4. **Water**: Challenges related to water supply, quality, and management, exacerbated by changing precipitation patterns and increased drought frequency.

5. **Public Health**: The potential health risks stemming fr

In [244]:
question_5 = questions[4]
response_5 = rag_chain.invoke({"input": question_5.lower()})
output_5 = f"### Output for Question 5\n\n**{question_5}**\n{response_5['answer']}\n"
print(f"### Output for Question 5\n\n**{question_5}**\n{response_5['answer']}\n")

### Output for Question 5

**For each identified climate impact, summarize the plan’s description of: The climate impact, the climate risks it poses to the planning jurisdiction, and areas or populations with the most climate vulnerability to the identified climate impact.**
The provided text does not explicitly enumerate specific climate impacts, climate risks, or vulnerable areas/populations. However, it does outline general strategies and actions the City of Los Angeles plans to implement to address climate change impacts. Based on the information available, I can summarize the implicit climate impacts and associated risks as follows:

1. **Urban Heat Island Effect**
   - **Climate Impact:** The plan mentions the development and implementation of urban heat island reduction plans and demonstration projects.
   - **Climate Risks:** Increased temperatures due to urban heat can lead to health risks, particularly for vulnerable populations such as the elderly, children, and those with p

In [245]:
question_6 = questions[5]
response_6 = rag_chain.invoke({"input": question_6.lower()})
output_6 = f"### Output for Question 6\n\n**{question_6}**\n{response_6['answer']}\n"
print(f"### Output for Question 6\n\n**{question_6}**\n{response_6['answer']}\n")

### Output for Question 6

**What resilience options does the plan identify? Summarize cost, timing, location, and important attributes.**
The plan identifies several resilience options organized around key themes: Energy, Transportation and Land Use, Ocean and Coastal Resources, Water, and Public Health. Here is a summary of these options along with their cost, timing, location, and important attributes:

1. **Energy**
   - **Cost**: Varies based on specific projects and technologies adopted.
   - **Timing**: Short-term to long-term, depending on the specific initiatives.
   - **Location**: City-wide, focusing on energy infrastructure and community access.
   - **Important Attributes**: Emphasis on renewable energy sources, energy efficiency, and energy storage solutions to enhance reliability during climate events.

2. **Transportation and Land Use**
   - **Cost**: Costs can range from moderate to high, depending on infrastructure changes.
   - **Timing**: Medium to long-term impleme

In [246]:
question_7 = questions[6]
response_7 = rag_chain.invoke({"input": question_7.lower()})
output_7 = f"### Output for Question 7\n\n**{question_7}**\n{response_7['answer']}\n"
print(f"### Output for Question 7\n\n**{question_7}**\n{response_7['answer']}\n")

### Output for Question 7

**What mitigation options does the plan identify? Summarize cost, timing, location, and important attributes.**
The provided excerpt does not explicitly detail specific mitigation options, but it does mention strategies and guiding principles that imply integration of climate adaptation and hazard mitigation within the General Plan and Zoning Code. Here’s a summary based on the information available:

1. **Guiding Principle for Land-Use Decisions**
   - **Cost**: N/A
   - **Timing**: Short Term
   - **Location**: Citywide
   - **Important Attributes**: Encourages land-use decisions that enhance resilience, aiding recovery from disasters and climate change impacts, while also addressing equity and long-term capital planning (N/A).

2. **Funding Pathways for Resilience**
   - **Cost**: Potentially significant investment; specific amounts not detailed.
   - **Timing**: Medium Term
   - **Location**: City and regional infrastructure
   - **Important Attributes**:

In [248]:
question_8 = questions[7]
response_8 = rag_chain.invoke({"input": question_8.lower()})
output_8 = f"### Output for Question 8\n\n**{question_8}**\n{response_8['answer']}\n"
print(f"### Output for Question 8\n\n**{question_8}**\n{response_8['answer']}\n")

### Output for Question 8

**What adaptation options does the plan identify? Summarize cost, timing, location, and important attributes.**
The plan identifies several adaptation options across different sectors, focusing on enhancing resilience to climate impacts. Here’s a summary of the identified adaptation options, including aspects of cost, timing, location, and important attributes:

1. **Land-Use Decisions**
   - **Cost**: Not specified
   - **Timing**: Short Term
   - **Location**: Citywide
   - **Important Attributes**: Incorporating climate adaptation, hazard mitigation, and recovery into the General Plan; leveraging long-term capital planning for infrastructure investment; promoting equity through improved land-use decisions (N/A).

2. **Energy Resilience**
   - **Cost**: Not specified
   - **Timing**: Not specified
   - **Location**: Citywide
   - **Important Attributes**: Addressing energy systems to improve resilience against climate impacts, ensuring the reliability of en

In [250]:
question_9 = questions[8]
response_9 = rag_chain.invoke({"input": question_9.lower()})
output_9 = f"### Output for Question 9\n\n**{question_9}**\n{response_9['answer']}\n"
print(f"### Output for Question 9\n\n**{question_9}**\n{response_9['answer']}\n")

### Output for Question 9

**What climate policies does the plan identify to advance climate justice? Summarize populations vulnerable to the identified climate impact.**
The plan identifies several climate policies aimed at advancing climate justice, which include:

1. **Incorporation of Equity into the General Plan**: The plan emphasizes the need to integrate climate adaptation, hazard mitigation, and recovery efforts with a focus on increasing equity in long-term capital planning for infrastructure investment (N/A).

2. **Development of Resilience Hubs**: A specific initiative is to develop resilience hubs in the city’s most vulnerable neighborhoods by 2028. This aims to provide resources and support to communities that are disproportionately affected by climate impacts (N/A).

3. **Access to Green Space**: The plan includes efforts to increase access to green and open spaces, particularly through investments along the Los Angeles River system and in underserved neighborhoods by 202

In [252]:
question_10 = questions[9]
response_10 = rag_chain.invoke({"input": question_10.lower()})
output_10 = f"### Output for Question 10\n\n**{question_10}**\n{response_10['answer']}\n"
print(f"### Output for Question 10\n\n**{question_10}**\n{response_10['answer']}\n")

### Output for Question 10

**Is there any climate policy identified in the plan that could be a potential maladaptation? Summarize the resilience, mitigation, or adaptation option.**
The plan identifies several actions aimed at climate adaptation and resilience, but there is a potential risk of maladaptation associated with the development of an Urban Heat Vulnerability Index and Mitigation Plan (Action 2.32). While the intention behind this action is to prepare for higher temperatures and more frequent extreme heat events, there are considerations that could lead to maladaptive outcomes if not carefully managed.

For instance, if the Urban Heat Vulnerability Index prioritizes certain neighborhoods for interventions without considering the socio-economic dynamics or existing environmental conditions, it could unintentionally exacerbate inequalities. Communities that are already vulnerable may not receive adequate resources or attention, leading to a situation where interventions fail 

In [253]:
question_11 = questions[10]
response_11 = rag_chain.invoke({"input": question_11.lower()})
output_11 = f"### Output for Question 11\n\n**{question_11}**\n{response_11['answer']}\n"
print(f"### Output for Question 11\n\n**{question_11}**\n{response_11['answer']}\n")

### Output for Question 11

**Summarize the plan's description of how the planning jurisdiction will monitor and evaluate progress on the implementation of resilience, mitigation, and adaptation options.**
The plan outlines that the City will measure and track outcomes related to resilience, mitigation, and adaptation options over the long term. This systematic monitoring will allow the City to focus on achieving progress toward resilience goals, ensuring that the initiatives integrated into the General Plan and Zoning Code effectively contribute to climate adaptation, hazard mitigation, and recovery efforts (Goal 8, Page number N/A). The emphasis is placed on continuous evaluation to ensure that land-use decisions and planning documents are aligned with the objectives of enhancing resilience for vulnerable communities and systems (Goal 8, Page number N/A).



In [254]:
question_12 = questions[11]
response_12 = rag_chain.invoke({"input": question_12.lower()})
output_12 = f"### Output for Question 12\n\n**{question_12}**\n{response_12['answer']}\n"
print(f"### Output for Question 12\n\n**{question_12}**\n{response_12['answer']}\n")

### Output for Question 12

**Generate a table that summarizes the following features from the document: Location/City Name, Population, Threats Identified, Measures Identified.**
Here is a table summarizing the features from the provided document:

| Feature                | Details                              |
|-----------------------|--------------------------------------|
| Location/City Name    | N/A (not specified in the document) |
| Population            | N/A (not specified in the document) |
| Threats Identified    | 1. Disasters (e.g., earthquakes)   |
|                       | 2. Climate change impacts (e.g., extreme heat) |
|                       | 3. Economic shifts                   |
| Measures Identified    | 1. Incorporate climate adaptation into the General Plan |
|                       | 2. Adopt emergency land-use tools   |
|                       | 3. Develop urban heat vulnerability index and mitigation plan |
|                       | 4. Neighborhood retrofi

In [262]:
merged_response = output_1 + output_2 + output_3 + output_4 + output_5 + output_6 + output_7 + output_8 + output_9 + output_10 + output_11 + output_12

In [264]:
with open("los_angeles_plan_manyqueries.md", "w") as file:
    file.write(merged_response)
print("Markdown document created: los_angeles_plan_manyqueries.md")

Markdown document created: los_angeles_plan_manyqueries.md
